In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re

In [2]:
def get_long_lat(location):
    coords = None
    driver.get("https://www.google.com/maps/search/{}".format(location))
    driver.implicitly_wait(3)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    meta_tag = soup.find_all("meta", itemprop="image")
    markers = re.split('markers=', str(meta_tag[0]))
    # print(str(len(markers)))
    # print(markers)
    if len(markers) > 1:
        x = soup.find('div', class_="qBF1Pd fontHeadlineSmall")
        name_ = re.search('>(.*)<', str(x)).group(1)
        # if name_.lower() == location_name.lower():
        #     coords = re.split('markers=|&amp;', str(meta_tag[0]))[4]
        coords = re.split('markers=|&amp;', str(meta_tag[0]))[4]
    else:
        y = re.split('center=|&amp;', str(meta_tag[0]))
        if len(y) > 1:
            coords = y[1]

    if coords is None:
        lat = long = None
    else:
        lat = float(re.split('%|C', coords)[0])
        long = float(re.split('%|C', coords)[2])

    return [lat, long]

In [3]:
df = pd.read_csv("input.csv", sep=',')

In [4]:
df_na = df.loc[df.latitude.isna()]
df_no_na = df.loc[~df.latitude.isna()]
df_na['isna'] = 1
df_no_na['isna'] = 0

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)

for i, r in tqdm(df_na.iterrows(), total=df_na.shape[0]):
    location_name = r['Nama Lokasi']
    df_na.loc[i, ['latitude', 'longitude']] = get_long_lat(location_name)

100%|██████████| 9/9 [00:14<00:00,  1.56s/it]


In [6]:
df_na

,No,Nama Lokasi,latitude,longitude,isna
0,1,Taman Impian Jaya Ancol,-6.128970,106.833307,1
1,2,TMII,-6.302039,106.889892,1
2,3,Ragunan,-6.296829,106.821286,1
3,4,Monumen Nasional,-6.175392,106.827153,1
4,5,Museum Nasional,-6.176402,106.821590,1
5,6,Museum Satria Mandala,-6.231829,106.818944,1
6,7,Museum Sejarah Jakarta,-6.135200,106.813300,1
7,8,Pelabuhan Sunda Kelapa Jakarta,-6.119402,106.812094,1
8,9,BPS,-6.165246,106.837351,1


In [7]:
# x = pd.concat([df_na, df_no_na])
# x = x.sort_index()
# x.to_csv("../data/univ-selindo-final_24082023_complete.csv", sep=';')